### Make a surface and domain file given a lat  lon 

In [1]:
from scipy.stats import qmc
import numpy as np
import xarray as xr
import csv
import pandas as pd
import os
import netCDF4 as nc4
import sys
from tempfile import TemporaryFile                                                                                                                                 
import argparse                                                                                                                                                     
import shutil                                                                                                                                                       
import tempfile 
import random


/global/homes/j/jneedham/.conda/envs/myenv/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Domain file

In [23]:
### BCI 
#site = 'bci'
#lat = 9.154300000000
#lon = -79.846100000000

### Manaus
site = 'manaus'
lat = -2.609
lon = -60.209

if lon < 0 : 
    lon = 360 + lon
    print(lon)

299.791


In [26]:
domain_full = '/global/cfs/cdirs/e3sm/inputdata/share/domains/domain.lnd.fv1.9x2.5_gx1v6.090206.nc'
# high resolution run
#domain_full = '/global/cfs/cdirs/e3sm/inputdata/share/domains/domain.lnd.r0125_gx1v6.191017.nc'
domain_full = xr.open_dataset(domain_full,  decode_times=False)

In [27]:
lon_mids = domain_full.xc[0,0:144]
lat_mids = domain_full.yc[0:96,0]

abslat = np.abs(lat_mids - lat)
abslon = np.abs(lon_mids - lon)
c = np.maximum(abslon, abslat)

([xloc],[yloc]) = np.where(c == np.min(c))

lat_new = lat_mids[yloc]
print(lat_new.values)
lon_new = lon_mids[xloc]
print(lon_new.values)

domain = domain_full.sel(ni=[xloc],nj=[yloc])

(array([120]), array([46]))
-2.84210526315798
300.0


#### Set frac to 1

In [12]:
domain['frac'].values = [[1.0]]

#### Save domain file

In [13]:
domain.to_netcdf('/global/cfs/cdirs/m2420/fates-tutorial-2024/fates-tutorial/met_data/{0}/domain_{0}_fates_tutorial.nc'.format(site))

### Surface data file

In [14]:
surfdata_full = '/global/cfs/cdirs/e3sm/inputdata/lnd/clm2/surfdata_map/surfdata_1.9x2.5_simyr2000_c180306.nc'
surfdata_full = xr.open_dataset(surfdata_full,  decode_times=False)

##### Subset by lat and lon

In [15]:
surfdata = surfdata_full.sel(lsmlon=[xloc],lsmlat=[yloc])

#### Set percent vegetation to 100%

In [16]:
new_urban = [[[0]],[[0]],[[0]]]
new_urban_array = xr.DataArray(new_urban, dims=['numurbl', 'lsmlat', 'lsmlon'],
                               coords = {'numurbl':surfdata['numurbl'],'lsmlat':surfdata['lsmlat'],'lsmlon':surfdata['lsmlon']})
surfdata['PCT_URBAN'] = new_urban_array

surfdata['PCT_LAKE'].values = [[0.0]]
surfdata['PCT_NATVEG'].values = [[100.0]]

#### Save surface file

In [17]:
surfdata.to_netcdf('/global/cfs/cdirs/m2420/fates-tutorial-2024/fates-tutorial/met_data/{0}/surfdata_{0}_fates_tutorial.nc'.format(site))